#　画像切り取りプログラム

In [ ]:
import cv2
import os
import glob

# ==========================================
# 設定エリア
# ==========================================
INPUT_FOLDER = 'img/plant9-'          # 元画像があるフォルダ
OUTPUT_FOLDER = 'img/cropped_images/plant9-' # 切り取った画像を保存するフォルダ
DISPLAY_HEIGHT = 1000             # 表示ウィンドウの高さ
# ==========================================

# グローバル変数
ix, iy = -1, -1 
ex, ey = -1, -1 
drawing = False 
img_resized = None 
scale_factor = 1.0 

# マウスイベント処理
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, ex, ey, drawing, img_resized

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        ex, ey = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img_resized.copy()
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 0, 255), 2)
            cv2.imshow('Image Cropper', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        ex, ey = x, y
        cv2.rectangle(img_resized, (ix, iy), (ex, ey), (0, 0, 255), 2)
        cv2.imshow('Image Cropper', img_resized)

def main():
    global img_resized, scale_factor, ix, iy, ex, ey

    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.[jJ][pP][gG]')))
    if not files:
        files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.png')))
    
    if not files:
        print(f"エラー: '{INPUT_FOLDER}' フォルダに画像が見つかりません。")
        return

    print(f"{len(files)} 枚の画像を処理します。")
    print("--------------------------------------------------")
    print("【操作方法】")
    print(" 1. マウスドラッグ : 新しく範囲を指定")
    print(" 2. 'c' キー      : 確定して保存＆次へ (前回の範囲を維持)")
    print(" 3. 'r' キー      : 範囲をリセット")
    print(" 4. 'q' キー      : 終了")
    print("--------------------------------------------------")

    # ウィンドウを作成
    window_name = 'Image Cropper'
    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, draw_rectangle)

    ix, iy, ex, ey = -1, -1, -1, -1

    for i, file_path in enumerate(files):
        filename = os.path.basename(file_path)
        img_original = cv2.imread(file_path)
        if img_original is None:
            continue
        
        # --- 表示用にリサイズ ---
        h, w = img_original.shape[:2]
        scale_factor = DISPLAY_HEIGHT / h
        new_w = int(w * scale_factor)
        new_h = int(h * scale_factor)
        
        img_resized_base = cv2.resize(img_original, (new_w, new_h))
        img_resized = img_resized_base.copy()

        # ★前回選択した範囲がある場合、赤枠を描画
        if ix != -1 and ex != -1:
            cv2.rectangle(img_resized, (ix, iy), (ex, ey), (0, 0, 255), 2)
        
        # ウィンドウタイトルにもファイル名を表示
        cv2.setWindowTitle(window_name, f"Editing: {filename}")
        
        cv2.imshow(window_name, img_resized)
        print(f"[{i+1}/{len(files)}] 表示中: {filename}")

        while True:
            k = cv2.waitKey(1) & 0xFF

            if k == ord('c'):
                x1_disp, y1_disp = min(ix, ex), min(iy, ey)
                x2_disp, y2_disp = max(ix, ex), max(iy, ey)

                if x2_disp - x1_disp > 0 and y2_disp - y1_disp > 0:
                    real_x1 = int(x1_disp / scale_factor)
                    real_y1 = int(y1_disp / scale_factor)
                    real_x2 = int(x2_disp / scale_factor)
                    real_y2 = int(y2_disp / scale_factor)

                    real_x1 = max(0, real_x1)
                    real_y1 = max(0, real_y1)
                    real_x2 = min(w, real_x2)
                    real_y2 = min(h, real_y2)

                    cropped_img = img_original[real_y1:real_y2, real_x1:real_x2]
                    save_path = os.path.join(OUTPUT_FOLDER, filename)
                    cv2.imwrite(save_path, cropped_img)
                    print(f" -> 保存完了")
                    break
                else:
                    print("範囲を選択してください。")

            elif k == ord('r'):
                # リセット時もファイル名は再描画
                img_resized = img_resized_base.copy()
                cv2.putText(img_resized, f"[{i+1}/{len(files)}] {filename}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                
                ix, iy, ex, ey = -1, -1, -1, -1
                cv2.imshow(window_name, img_resized)
                print(" -> リセット")

            elif k == ord('q'):
                cv2.destroyAllWindows()
                return

    cv2.destroyAllWindows()
    print("完了しました。")

if __name__ == "__main__":
    main()

# 前処理

In [ ]:
import cv2
import numpy as np
import os
import glob

# 設定パラメータ
INPUT_FOLDER = 'img/cropped_images/plant9-'   # 入力元フォルダ
OUTPUT_BASE = 'img/processed_blue'            # 出力先ベースフォルダ
DIR_NAME_SUB = 'subtracted_plant9-'           # 差分演算結果保存先

def main():
    # 出力ディレクトリパスの生成
    out_sub_dir = os.path.join(OUTPUT_BASE, DIR_NAME_SUB)

    # ディレクトリが存在しない場合は作成
    if not os.path.exists(out_sub_dir):
        os.makedirs(out_sub_dir)

    # 画像ファイルの取得 (jpg, png対応)
    files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.[jJ][pP][gG]')))
    if not files:
        files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.png')))
    
    if not files:
        print(f"Error: 画像が見つかりません -> {INPUT_FOLDER}")
        return

    print(f"処理対象: {len(files)} 枚")
    print("-" * 50)

    for i, file_path in enumerate(files):
        filename = os.path.basename(file_path)
        img = cv2.imread(file_path)
        
        if img is None:
            continue

        # 1. チャンネル分解 (Blue, Green, Red)
        b, g, r = cv2.split(img)

        # -------------------------------------------------
        # 色差強調ロジック (R + G) - B
        # -------------------------------------------------
        # 演算時のオーバーフローを防ぐため int16 にキャスト
        r_16 = r.astype(np.int16)
        g_16 = g.astype(np.int16)
        b_16 = b.astype(np.int16)
        
        # 演算実行: 赤と緑の和から青成分を引く
        # これにより青みがかった背景(B成分大)を抑制し、根(R,G成分大)を強調する
        subtracted = (r_16 + g_16) - b_16
        
        # 0～255の範囲にクリッピングして uint8 に戻す
        img_sub = np.clip(subtracted, 0, 255).astype(np.uint8)

        # -------------------------------------------------
        # 画像の保存
        # -------------------------------------------------
        cv2.imwrite(os.path.join(out_sub_dir, filename), img_sub)

        # 進捗ログ (10枚ごと)
        if (i + 1) % 10 == 0 or (i + 1) == len(files):
            print(f"[{i+1}/{len(files)}] Processed: {filename}")

    print("-" * 50)
    print("全処理完了")
    print(f"保存先: {out_sub_dir}")

if __name__ == "__main__":
    main()

# データ作成プログラム

In [ ]:
import cv2
import numpy as np
import pandas as pd
import glob
import os

# ==========================================
# 設定パラメータ
# ==========================================
INPUT_FOLDER = 'img/processed_blue/subtracted_plant9-'  # 入力画像フォルダ
OUTPUT_FILE = 'root_area_data_plant9-.xlsx'           # 結果保存ファイル名

DISPLAY_SCALE = 0.7          # 表示倍率
INITIAL_BRUSH_SIZE = 20      # 初期ブラシサイズ
AUTO_THRESHOLD = 70           # 自動検出の二値化閾値
MAX_UNDO = 10                # Undo履歴の最大保持数

# グローバル変数初期化
drawing = False
erasing = False
mask_roi = None              # 探索範囲マスク (青)
mask_root = None             # 根領域マスク (赤)
current_mode = 0             # 0: ROI選択モード, 1: 根編集モード
current_brush_size = INITIAL_BRUSH_SIZE

# マウス座標・Undo履歴
mouse_curr_x, mouse_curr_y = 0, 0
undo_stack_roi = []
undo_stack_root = []

def save_undo_state():
    """現在のマスク状態を履歴に保存"""
    global undo_stack_roi, undo_stack_root
    
    if current_mode == 0:
        if mask_roi is not None:
            undo_stack_roi.append(mask_roi.copy())
            if len(undo_stack_roi) > MAX_UNDO:
                undo_stack_roi.pop(0)
    else:
        if mask_root is not None:
            undo_stack_root.append(mask_root.copy())
            if len(undo_stack_root) > MAX_UNDO:
                undo_stack_root.pop(0)

def perform_undo():
    """直前の操作を取り消す"""
    global mask_roi, mask_root
    
    if current_mode == 0:
        if undo_stack_roi:
            mask_roi[:] = undo_stack_roi.pop()
            print("Undo: ROI selection")
    else:
        if undo_stack_root:
            mask_root[:] = undo_stack_root.pop()
            print("Undo: Root editing")

def on_mouse(event, x, y, flags, param):
    """マウスイベント処理"""
    global drawing, erasing, mouse_curr_x, mouse_curr_y
    
    # UI制御用にマウス位置を更新
    mouse_curr_x, mouse_curr_y = x, y
    
    # 画像座標へ変換
    real_x = int(x / DISPLAY_SCALE)
    real_y = int(y / DISPLAY_SCALE)
    
    if mask_roi is None:
        return
    
    h, w = mask_roi.shape[:2]
    if not (0 <= real_x < w and 0 <= real_y < h):
        return

    # 操作対象のマスクを選択
    target_mask = mask_roi if current_mode == 0 else mask_root
    draw_r = current_brush_size
    erase_r = max(2, int(current_brush_size / 2))

    # 左クリック: 描画
    if event == cv2.EVENT_LBUTTONDOWN:
        save_undo_state()
        drawing = True
        cv2.circle(target_mask, (real_x, real_y), draw_r, 255, -1)
    
    # 右クリック: 消去
    elif event == cv2.EVENT_RBUTTONDOWN:
        save_undo_state()
        erasing = True
        cv2.circle(target_mask, (real_x, real_y), erase_r, 0, -1)
    
    # ドラッグ中
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(target_mask, (real_x, real_y), draw_r, 255, -1)
        elif erasing:
            cv2.circle(target_mask, (real_x, real_y), erase_r, 0, -1)
            
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
    elif event == cv2.EVENT_RBUTTONUP:
        erasing = False

def auto_detect_in_roi(img_gray):
    """ROI内部で根を自動検出する"""
    global mask_root
    
    # 二値化処理
    _, binary = cv2.threshold(img_gray, AUTO_THRESHOLD, 255, cv2.THRESH_BINARY)
    
    # ROI(青マスク)内のみ抽出
    detected = cv2.bitwise_and(binary, binary, mask=mask_roi)
    
    # ノイズ除去 (オープニング処理)
    kernel = np.ones((3, 3), np.uint8)
    detected = cv2.morphologyEx(detected, cv2.MORPH_OPEN, kernel, iterations=1)
    
    mask_root[:] = detected[:]

def main():
    global mask_roi, mask_root, current_mode, drawing, erasing
    global undo_stack_roi, undo_stack_root, current_brush_size

    # 画像ファイルの読み込み
    files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.[jJ][pP][gG]')))
    if not files:
        files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.png')))
    
    if not files:
        print(f"Error: 画像が見つかりません -> {INPUT_FOLDER}")
        return

    # 初回初期化
    first_img = cv2.imread(files[0], cv2.IMREAD_GRAYSCALE)
    h, w = first_img.shape[:2]
    mask_roi = np.zeros((h, w), np.uint8)
    mask_root = np.zeros((h, w), np.uint8)

    window_name = "Semi-Automatic Measurement Tool"
    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, on_mouse)

    print("-" * 50)
    print(" [操作方法]")
    print("  Left Drag  : 塗る")
    print("  Right Drag : 消す")
    print("  [m] : モード切替 / 自動検出実行")
    print("  [z] : Undo")
    print("  [b] : 前の画像に戻る")
    print("  [q] : 終了")
    print("-" * 50)

    results = []
    idx = 0
    
    while idx < len(files):
        file_path = files[idx]
        filename = os.path.basename(file_path)
        img_curr = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        
        if img_curr is None:
            idx += 1
            continue
            
        img_disp_base = cv2.cvtColor(img_curr, cv2.COLOR_GRAY2BGR)

        # 画像サイズが変わった場合のマスク追従
        if mask_roi.shape != img_curr.shape:
            mask_roi = cv2.resize(mask_roi, (img_curr.shape[1], img_curr.shape[0]), interpolation=cv2.INTER_NEAREST)
            mask_root = cv2.resize(mask_root, (img_curr.shape[1], img_curr.shape[0]), interpolation=cv2.INTER_NEAREST)

        # 履歴リセット (画像切り替え時)
        undo_stack_roi = []
        undo_stack_root = []

        while True:
            # 表示用リサイズ
            disp_h = int(h * DISPLAY_SCALE)
            disp_w = int(w * DISPLAY_SCALE)
            view_img = cv2.resize(img_disp_base, (disp_w, disp_h))
            
            # オーバーレイ描画
            if current_mode == 0:
                # Mode 1: ROI選択 (青)
                resized_mask = cv2.resize(mask_roi, (disp_w, disp_h))
                overlay = np.zeros_like(view_img)
                overlay[resized_mask > 0] = (255, 0, 0)
                final_view = cv2.addWeighted(view_img, 1.0, overlay, 0.3, 0)
                
                status_txt = "Mode 1: ROI Select (Blue)"
                next_txt = "Press [m] -> Auto Detect"
                status_color = (255, 0, 0)
            else:
                # Mode 2: 根領域修正 (赤)
                resized_mask = cv2.resize(mask_root, (disp_w, disp_h))
                overlay = np.zeros_like(view_img)
                overlay[resized_mask > 0] = (0, 0, 255)
                final_view = cv2.addWeighted(view_img, 1.0, overlay, 0.5, 0)
                
                status_txt = "Mode 2: Edit Root (Red)"
                next_txt = "Press [m] -> Save & Next"
                status_color = (0, 0, 255)

            # UIテキスト表示制御 (マウスが重なると消える)
            ui_top_area = (0, 0, 500, 120)
            ui_btm_area = (0, disp_h - 40, 300, disp_h)
            
            is_hover_top = (ui_top_area[0] <= mouse_curr_x <= ui_top_area[2] and 
                            ui_top_area[1] <= mouse_curr_y <= ui_top_area[3])
            is_hover_btm = (ui_btm_area[0] <= mouse_curr_x <= ui_btm_area[2] and 
                            ui_btm_area[1] <= mouse_curr_y <= ui_btm_area[3])

            if not (is_hover_top or is_hover_btm):
                cv2.putText(final_view, f"[{idx+1}/{len(files)}] {filename}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(final_view, status_txt, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
                cv2.putText(final_view, next_txt, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(final_view, f"Brush: {current_brush_size}", (10, disp_h - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            cv2.imshow(window_name, final_view)
            
            key = cv2.waitKey(10) & 0xFF

            # [m] モード進行
            if key == ord('m'):
                if current_mode == 0:
                    current_mode = 1
                    auto_detect_in_roi(img_curr)
                else:
                    # 面積算出と保存
                    area = cv2.countNonZero(mask_root)
                    results.append({'filename': filename, 'area_pixels': area})
                    print(f"Recorded: {filename} -> Area: {area}")
                    
                    idx += 1
                    current_mode = 0
                    drawing = False
                    erasing = False
                    break
            
            # [ [ ] ] ブラシサイズ変更
            elif key == ord('['):
                current_brush_size = max(2, current_brush_size - 2)
            elif key == ord(']'):
                current_brush_size = min(100, current_brush_size + 2)
            
            # [z] Undo
            elif key == ord('z'):
                perform_undo()
            
            # [b] Back
            elif key == ord('b'):
                if idx > 0:
                    idx -= 1
                    current_mode = 0
                    break
            
            # [r] Reset Mask
            elif key == ord('r'):
                if current_mode == 0: mask_roi[:] = 0
                else: mask_root[:] = 0
            
            # [q] Quit
            elif key == ord('q'):
                idx = len(files)
                break

    cv2.destroyAllWindows()

    # 結果の保存
    if results:
        df = pd.DataFrame(results)
        df['growth_rate(%)'] = df['area_pixels'].pct_change().fillna(0) * 100
        df.to_excel(OUTPUT_FILE, index=False)
        print(f"\nAll Done. Results saved to: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

#　エクセルデータ補正

In [ ]:
import pandas as pd
import os

# ==========================================
# 設定パラメータ
# ==========================================
INPUT_FILE = 'root_area_data_plant9-.xlsx'            # 入力データ (Excel)
OUTPUT_FILE = 'root_area_data_corrected_plant9-.xlsx'  # 補正後データ保存先

def fix_data():
    """
    時系列データの単調増加補正を行う関数
    根は生物学的に短期間で縮小しないため、計測ノイズによる減少を補正する
    """
    
    # ファイル存在確認
    if not os.path.exists(INPUT_FILE):
        print(f"Error: ファイルが見つかりません -> {INPUT_FILE}")
        return

    # Excel読み込み
    try:
        df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"Error: ファイル読み込み失敗 ({e})")
        return
    
    print("データ補正処理を開始します...")
    
    corrected_areas = []
    max_area_so_far = 0
    correction_count = 0
    
    # 1行ずつスキャンして補正
    for i, row in df.iterrows():
        current_area = row['area_pixels']
        
        # 現在の値がこれまでの最大値以上であれば更新
        if current_area >= max_area_so_far:
            max_area_so_far = current_area
            corrected_areas.append(current_area)
        else:
            # 値が減少している場合（ノイズ判定）
            # 直近の最大値で値を維持する
            # print(f" -> Corrected: {row['filename']} ({current_area} -> {max_area_so_far})")
            corrected_areas.append(max_area_so_far)
            correction_count += 1
            
    # データフレームの値を更新
    df['area_pixels'] = corrected_areas
    
    # 成長率（変化率）を再計算
    df['growth_rate(%)'] = df['area_pixels'].pct_change().fillna(0) * 100
    
    # 保存
    df.to_excel(OUTPUT_FILE, index=False)
    
    print("-" * 50)
    print(f"処理完了")
    print(f"補正箇所数: {correction_count}")
    print(f"保存先: {OUTPUT_FILE}")

if __name__ == "__main__":
    fix_data()

#　１ピクセル当たりの面積算出（㎟）

In [ ]:
import cv2
import math
import os

# 設定パラメータ
TARGET_IMAGE = 'img/plant6-/20251206_182331.jpg'  # 計測対象の画像パス

# グローバル変数
points = []
img_display = None

def click_event(event, x, y, flags, param):
    """
    マウスクリックイベントハンドラ
    2点をクリックして直線を描画する
    """
    global points, img_display

    if event == cv2.EVENT_LBUTTONDOWN:
        # 2点未満の場合のみ座標を記録
        if len(points) < 2:
            points.append((x, y))
            
            # クリック位置に赤点を描画
            cv2.circle(img_display, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow('Scale Calibration', img_display)
            print(f"Point {len(points)}: ({x}, {y})")

            # 2点揃ったら緑線を描画
            if len(points) == 2:
                cv2.line(img_display, points[0], points[1], (0, 255, 0), 2)
                cv2.imshow('Scale Calibration', img_display)
                print(" -> 2点選択完了。任意のキーを押して計算に進んでください。")

def main():
    global img_display
    
    # 画像ファイルの存在確認
    if not os.path.exists(TARGET_IMAGE):
        print(f"Error: 画像が見つかりません -> {TARGET_IMAGE}")
        return

    # 画像読み込み
    img_display = cv2.imread(TARGET_IMAGE)
    
    # ウィンドウ設定
    window_name = 'Scale Calibration'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(window_name, click_event)

    print("-" * 50)
    print(" [操作手順]")
    print("  1. 既知の長さ（定規など）の始点と終点をクリック")
    print("  2. 緑色の線が表示されたら、任意のキーを押して確定")
    print("-" * 50)

    cv2.imshow(window_name, img_display)
    
    # キー入力待ち
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # 2点選択されていない場合は終了
    if len(points) != 2:
        print("Error: 2点が選択されませんでした。")
        return

    # 1. ピクセル距離の計算 (ユークリッド距離)
    p1, p2 = points
    distance_px = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
    print(f"画像上の距離: {distance_px:.2f} px")

    # 2. 実寸法の入力
    try:
        real_length_cm = float(input("実際の長さ(cm)を入力してください: "))
    except ValueError:
        print("Error: 数値を入力してください。")
        return

    # 3. スケール換算係数の算出
    # 1ピクセルあたりの長さ (cm/px)
    cm_per_pixel = real_length_cm / distance_px
    
    # 1ピクセルあたりの面積 (cm^2/px)
    area_per_pixel = cm_per_pixel ** 2

    print("-" * 50)
    print(" [計算結果]")
    print(f"  実際の長さ      : {real_length_cm} cm")
    print(f"  変換係数 (長さ) : {cm_per_pixel:.6f} cm/px")
    print(f"  変換係数 (面積) : {area_per_pixel:.8f} cm^2/px")
    print("-" * 50)
    print(" ※この '変換係数(面積)' を解析プログラムの設定値に使用してください。")

if __name__ == "__main__":
    main()

#　ピクセル→面積に変換

In [ ]:
import pandas as pd
import os

# ==========================================
# 設定パラメータ
# ==========================================
INPUT_FILE = 'root_area_data_corrected_plant9-.xlsx'   # 入力ファイル (補正済みデータ)
OUTPUT_FILE = 'root_area_data_mm2_plant9-.xlsx'      # 出力ファイル (mm2換算データ)

# キャリブレーション設定 (計測した基準値)
IMAGE_WIDTH_PX = 1519.16   # 基準となる長さ (pixels)
REAL_WIDTH_CM = 10.0       # その長さの実寸 (cm)
# ==========================================

def convert_pixels_to_mm2():
    """
    ピクセル単位の面積を実単位(mm^2)に変換するスクリプト
    """
    
    # 入力ファイルの確認
    if not os.path.exists(INPUT_FILE):
        print(f"Error: ファイルが見つかりません -> {INPUT_FILE}")
        return

    # 1. 変換係数の算出
    # 単位を mm に統一 (1cm = 10mm)
    real_width_mm = REAL_WIDTH_CM * 10.0

    # 1ピクセルあたりの長さ (mm/px)
    mm_per_pixel = real_width_mm / IMAGE_WIDTH_PX
    
    # 1ピクセルあたりの面積 (mm^2/px)
    area_per_pixel = mm_per_pixel ** 2
    
    print("-" * 50)
    print(" [変換設定]")
    print(f"  基準ピクセル : {IMAGE_WIDTH_PX} px")
    print(f"  実寸 (mm)    : {real_width_mm} mm")
    print(f"  変換係数     : {area_per_pixel:.8f} mm^2/px")
    print("-" * 50)

    # 2. データ読み込みと変換処理
    try:
        df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"Error: Excel読み込み失敗 ({e})")
        return

    print("面積データの変換を実行中...")

    # 'area_pixels' 列の存在確認
    if 'area_pixels' in df.columns:
        # ピクセル面積に係数を掛けて mm^2 を算出
        df['area_mm2'] = df['area_pixels'] * area_per_pixel
    else:
        print("Error: 'area_pixels' 列が見つかりません。")
        return

    # 成長率(%)の再計算
    df['growth_rate(%)'] = df['area_mm2'].pct_change().fillna(0) * 100

    # 3. 列の並び替えと保存
    # 見やすいように filename, area_mm2 を先頭に配置
    priority_cols = ['filename', 'area_mm2', 'area_pixels', 'growth_rate(%)']
    other_cols = [c for c in df.columns if c not in priority_cols]
    df = df[priority_cols + other_cols]

    # ファイル保存
    df.to_excel(OUTPUT_FILE, index=False)
    
    print("-" * 50)
    print("処理完了")
    print(f"  最大面積 : {df['area_mm2'].max():.2f} mm^2")
    print(f"  保存先   : {OUTPUT_FILE}")

if __name__ == "__main__":
    convert_pixels_to_mm2()